In [1]:
import numpy as np
import operator

In [2]:
def topo_sort_v1(a):
    done = []
    def visit(val):
        if val.children:
            for i in val.children:
                visit(i)
        if val not in done:
            done.append(val)
        return done[::-1]
    return visit(a)

In [3]:
def topo_sort_v2(a):
    done = []
    def visit(val):
        if val not in done:
            if val.children:
                for i in val.children:
                    visit(i)
            done.append(val)
        return done[::-1]
    return visit(a)

In [4]:
class Value:
    def __init__(self, value, label='', children=(), op='', name=''):
        self.value = value
        self.label = label
        self.grad = 0.0
        self.children = set(children)
        self.op = op
        self._backward = lambda x=None: None
        self._forward = lambda x=None: None
        self.name = name
        self.isconstant = False
        # print(f"{f'"{self.name}" = ' if self.name else f'{self.label} = '}{self.value:.4f}")

    def __repr__(self):
        return f"{f'"{self.name}" = ' if self.name else ''}{f'{self.label} = ' if not self.isconstant else ''}{round(self.value, 4)}, grad = {round(self.grad, 4)}"

    
    # UNARY OPERATORS
    def __pos__(self):
        return self

    def __neg__(self):
        return (-1) * self

    
    # ADDITION
    def __add__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            other = Value(other, f'{other}', name=f'const {other}')
            other.isconstant = True
            
        res = Value(self.value + other.value, label=f'[{self.label} + {other.label}]', children=(self, other), op='+')
        def _backward(do_print=True):
            self.grad += res.grad
            other.grad += res.grad
            if do_print:
                print(self)
                print(other)
            
        def _forward():
            res.value = self.value + other.value

        res._forward = _forward
        res._backward = _backward
        return res

    def __radd__(self, other):
        return self + other

    
    # SUBTRACTION
    def __sub__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            other = Value(other, f'{other}', name=f'const {other}')
            other.isconstant = True

        res = self + -other
        res.label = f'[{self.label} - {other.label}]'
        res.op = '-'
        return res

    def __rsub__(self, other):
        other = Value(other, f'{other}', name=f'const {other}')
        other.isconstant = True

        res = -self + other
        res.label = f'[{other.value} - {self.label}]'
        return res

    
    # MULTIPLICATION
    def __mul__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            other = Value(other, f'{other}', name=f'const {other}')
            other.isconstant = True
            
        res = Value(self.value * other.value, label=f'({self.label} * {other.label})', children=(self, other), op='*')
        def _backward(do_print=True):
            self.grad += other.value * res.grad
            other.grad += self.value * res.grad
            if do_print:
                print(self)
                print(other)

        def _forward():
            res.value = self.value * other.value
            
        res._forward = _forward
        res._backward = _backward
        return res

    def __rmul__(self, other):
        return self * other
        

    # EXPONENTIAL OPERATOR
    def do_power(self, y):
        result = Value(self.value ** y.value, label=f'{self.label} ^ {y.label}', children=(self, y), op='^')

        def _backward(do_print=True):
            self.grad += self.value ** (y.value - 1) * (y.value) * result.grad
            y.grad += np.log(self.value) * result.value * result.grad
            if do_print:
                print(self)
                print(y)

        def _forward():
            result.value = self.value ** y.value

        result._forward = _forward
        result._backward = _backward
        return result

    def __pow__(self, other):  # for x**y and x**2
        if isinstance(other, int) or isinstance(other, float):
            other = Value(other, f'{other}', name=f'const {other}')
            other.isconstant = True
        return self.do_power(other)
        
    def __rpow__(self, other):  # for 2**x with: self = x, other = 2
        other = Value(other, f'{other}', name=f'const {other}')
        other.isconstant = True
        return other.do_power(self)
        
    
    # DIVISON        
    def __truediv__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            other = Value(other, f'{other}', name=f'const {other}')
            other.isconstant = True
            
        res = self * other**(-1)
        res.label = f'({self.label} / {other.label})'
        res.op = '/'
        return res

    def __rtruediv__(self, other):
        other = Value(other, f'{other}', name=f'const {other}')
        other.isconstant = True

        res = self**(-1) * other
        res.label = f'({other.label} / {self.label})'
        res.op = '/'
        return res

    
    # FORWARD PROPAGATION
    def forward(self):
        order = topo_sort_v2(self)[::-1]
        for i in order:
            i._forward()

    
    # BACKWARD PROPAGATION
    def backward(self, do_print=True):
        order = topo_sort_v2(self)
        for i in order:
            i.grad = 0
        self.grad=1
        if do_print:
            print(self)
        for i in order:
            i._backward(do_print)

In [5]:
def gradient_descent(rate, iterations, node, do_print=True):
    node.backward(do_print)
    for i in range(iterations):
        order = topo_sort_v2(node)
        for n in order:
            if not n.children and not n.isconstant:
                n.value -= rate*n.grad
        if do_print:
            print(f'\ni = {i}')
        node.forward()
        node.backward(do_print)
    return node

In [6]:
# TEST ALL THE DIFFERENT OPERATIONS AND COMBINATIONS
x = Value(4, 'x')
y = Value(3, 'y')
print(); print('3 + x'); print(3 + x)
print(); print('x + 3'); print(x + 3)
print(); print('x + y'); print(x + y)
print()
print(); print('3 - x'); print(3 - x)
print(); print('x - 3'); print(x - 3)
print(); print('x - y'); print(x - y)
print()
print(); print('3 * x'); print(3 * x)
print(); print('x * 3'); print(x * 3)
print(); print('x * y'); print(x * y)
print()
print(); print('3 / x'); print(3 / x)
print(); print('x / 3'); print(x / 3)

print()
print()

print(); print('x / y'); z = x / y; print(z); z.backward(False); print(f'{x}\n{y}')
print(); print('y / x'); z = y / x; print(z); z.backward(False); print(f'{x}\n{y}')
print(); print('x ^ y'); z = x ** y; print(z); z.backward(False); print(f'{x}\n{y}')
print(); print('y ^ x'); z = y ** x; print(z); z.backward(False); print(f'{x}\n{y}')
print(); print('3 ^ x'); z = 3 ** x; print(z); z.backward(False); print(x)


3 + x
[x + 3] = 7, grad = 0.0

x + 3
[x + 3] = 7, grad = 0.0

x + y
[x + y] = 7, grad = 0.0


3 - x
[3 - x] = -1, grad = 0.0

x - 3
[x - 3] = 1, grad = 0.0

x - y
[x - y] = 1, grad = 0.0


3 * x
(x * 3) = 12, grad = 0.0

x * 3
(x * 3) = 12, grad = 0.0

x * y
(x * y) = 12, grad = 0.0


3 / x
(3 / x) = 0.75, grad = 0.0

x / 3
(x / 3) = 1.3333, grad = 0.0



x / y
(x / y) = 1.3333, grad = 0.0
x = 4, grad = 0.3333
y = 3, grad = -0.4444

y / x
(y / x) = 0.75, grad = 0.0
x = 4, grad = -0.1875
y = 3, grad = 0.25

x ^ y
x ^ y = 64, grad = 0.0
x = 4, grad = 48
y = 3, grad = 88.7228

y ^ x
y ^ x = 81, grad = 0.0
x = 4, grad = 88.9876
y = 3, grad = 108

3 ^ x
3 ^ x = 81, grad = 0.0
x = 4, grad = 88.9876


In [7]:
# TRY BACKPROPAGATION ON EXAMPLE FROM VIDEO
a = Value(2, 'a', name='a')
b = Value(-3, 'b', name='b')
e = a * b
c = Value(10, 'c', name='c')
d = e + c
f = Value(-2, 'f', name='f')
L = d * f
a.name = 'a'
d.name = 'd'
e.name = 'e'
L.name = 'L'

for i in topo_sort_v2(L): print(i)  # see all the values (with gradients set to 0)

"L" = ([(a * b) + c] * f) = -8, grad = 0.0
"d" = [(a * b) + c] = 4, grad = 0.0
"c" = c = 10, grad = 0.0
"e" = (a * b) = -6, grad = 0.0
"a" = a = 2, grad = 0.0
"b" = b = -3, grad = 0.0
"f" = f = -2, grad = 0.0


In [8]:
L.grad = 1  # manually backpropagate
L._backward()
f._backward()
d._backward()
e._backward()

"d" = [(a * b) + c] = 4, grad = -2.0
"f" = f = -2, grad = 4.0
"e" = (a * b) = -6, grad = -2.0
"c" = c = 10, grad = -2.0
"a" = a = 2, grad = 6.0
"b" = b = -3, grad = -4.0


In [9]:
for i in topo_sort_v1(L): print(i)  # check all values to see if all gradients changed

"L" = ([(a * b) + c] * f) = -8, grad = 1
"d" = [(a * b) + c] = 4, grad = -2.0
"c" = c = 10, grad = -2.0
"e" = (a * b) = -6, grad = -2.0
"a" = a = 2, grad = 6.0
"b" = b = -3, grad = -4.0
"f" = f = -2, grad = 4.0


In [10]:
L.backward()  # automatically backpropagate (and get the same results)

"L" = ([(a * b) + c] * f) = -8, grad = 1
"d" = [(a * b) + c] = 4, grad = -2
"f" = f = -2, grad = 4
"e" = (a * b) = -6, grad = -2
"c" = c = 10, grad = -2
"a" = a = 2, grad = 6
"b" = b = -3, grad = -4


In [11]:
a.value = 5.0  # test forward propagation: set a = 0 and check how that affects L
L.forward()
L

"L" = ([(a * b) + c] * f) = 10.0, grad = 1

In [12]:
x = Value(5.59, 'x', name='x')
y = Value(5.59, 'y', name='y')
z = Value(5.59, 'z', name='z')
f = 4 + (x * y) ** 2 / z + 4  # make up a function
f.name = 'f'

for i in topo_sort_v2(f): print(i)  # check all of the parts it consists of (in needed order)

"f" = [[((x * y) ^ 2 / z) + 4] + 4] = 182.6769, grad = 0.0
"const 4" = 4, grad = 0.0
[((x * y) ^ 2 / z) + 4] = 178.6769, grad = 0.0
((x * y) ^ 2 / z) = 174.6769, grad = 0.0
(x * y) ^ 2 = 976.4438, grad = 0.0
(x * y) = 31.2481, grad = 0.0
"x" = x = 5.59, grad = 0.0
"y" = y = 5.59, grad = 0.0
"const 2" = 2, grad = 0.0
z ^ -1 = 0.1789, grad = 0.0
"z" = z = 5.59, grad = 0.0
"const -1" = -1, grad = 0.0
"const 4" = 4, grad = 0.0


In [13]:
gradient_descent(0.1, 1000, f, do_print=False)  # and gradient descent

"f" = [[((x * y) ^ 2 / z) + 4] + 4] = 8.0, grad = 1

In [14]:
print(f.value)  # get the correct result

8.000049322535123
